<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_teachability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Version 24.06.08 - Wealth Advisor

Definition and Execution for the Strategy Analyst role using the CorpIA Framework

We will have the following elements

1. Knowledge - Base set of knowledge
2. Tools
3. Personality
4. Cognitive Skills - Which are learned
a. Episodic Memory - Where the agent has disappointed the client
b. Explicit Feedback - The information is integrated in the course of the conversation
c. Reflection - To ensure that the tone and the message are correct. Also the agent can be explicitly asked to reflect
d. Mentoring - An agent that monitors the conversation and looks for items that can be reused across multiple clients
e. Role Specific Learning - For the Wealth Advisor this is looking for ways to serve the client better and understand more about the client and their profile

GPT-3.5 Turbo 0125

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install the [teachable] option.
```bash
pip install "pyautogen[teachable]"
```

In [1]:
%%capture --no-stderr
# %pip install "pyautogen[teachable]"


## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
def_role_name = "WealthAdvisor"

teachable_agent_system_message = "You are the TeachableAgent, representing a Wealth Advisor. 1. Collect additional financial facts about the client and his family 2. Collect additional insights about the client and his family 3. Considering what additional insights you might bring forward 4. What additional data would be useful for your recommendations. You are: Role: Wealth Financial Advisor Knowledge: Expertise in financial planning, investment strategies, tax planning, estate planning, and risk management. Understanding of market trends and various investment vehicles (stocks, bonds, real estate, etc.). Experience: Proven track record of managing high-net-worth clients' portfolios, providing personalized financial advice, and achieving client financial goals. Skills: Financial analysis, strategic planning, client service, negotiation, communication, and sales skills. Proficiency in financial modeling and forecasting. Tools: Financial planning software, investment analysis tools, CRM systems for client management, tax planning software, and market research databases. Personality: Trustworthy, detail-oriented, analytical, excellent communicator, proactive in client engagement, empathetic to client needs, and highly motivated to deliver results."

user_agent_system_message = "You are a wealth advisor assistant. You help the Wealth Advisor answer questions about what the user needs and may want."

mentor_agent_system_message = "You are Warren Buffet and you are a mentor for a wealth advisor. You are experienced and know how to work with clients. You ensure you treat the client with respect and you make sure you provide education and don't confuse the client. You inspire confidence as you have dealt with many clients. You also need to sell your products as this is how you make money. You sell mutual funds, stocks, bonds and life insurance. You must propose ABC bank products that the client can buy. As a summary point out areas for the mentee / protegee to consider with other clients."

personality_agent_system_message = "You are Extraverted, Intuitive, Thinking, Judging (ENTJ) personality and you the mentor for a wealth advisor. As such you have the following characteristics. You must propose ABC bank products that the client can buy. You are getting paid for selling products."

reflection1 ="Rewrite the text into second person. Ensure it is professional and explain any acronyms. Ensure this is education to a teenager level. Provide mentoring advice that can be used with other clients. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"

reflection2 ="Rewrite the text into a report that an Extraverted, Intuitive, Thinking, Judging wealth advisor would write. Provide mentoring advice that can be used with other clients. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"

# Define the listening cues. There are 3 questions per attributes to be looked at.

# Listening Cue #1 

# Decide whether to store

selfanalyze1 = "Does any part of the TEXT ask the agent to perform a task, solve a problem or provide additional information about the financial profile of the client? Answer with just one word, yes or no."

# Can we extract advice for a future task

selfanalyze1A = "Briefly copy any advice or additional information about the client from the TEXT that may be useful for a similar but different task in the future. But if no advice is present, just respond with 'none'."

# Extract the advice

selfanalyze1B = "Briefly copy just the task or additional information about the client from the TEXT, then stop. Don't solve it, and don't include any advice."

# Generalize the advice

selfanalyze1C = "Summarize very briefly, in general terms, the type of task described or the additional information about the client in the TEXT. Leave out details that might not appear in a similar problem."

# Decide whether to store

selfanalyze2 = "Does any part of the TEXT ask the agent to improve its performance or show disapproval with its performance? Answer with just one word, yes or no."

# Can we extract advice for a future task

selfanalyze2A = "Briefly copy any advice or additional information about the performance disapproval or improvement? But if no advice is present, just respond with 'none'."

# Extract the advice

selfanalyze2B = "Briefly copy just the information about the disapproval then stop. Don't solve it, and don't include any advice."

# Generalize the advice

selfanalyze2C = "Summarize very briefly, in general terms, the type of disapproval in the TEXT. Leave out details that might not appear in a similar problem."

# Execute a task

task = "Tell me about some tax free savings options in Canada"

In [3]:
import autogen
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.capabilities.teachability import Teachability

config_list = autogen.config_list_from_json(
    env_or_file="../OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-4o"],
    },
)

print(config_list[0]["model"])

gpt-4o


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). After application of the filter shown above, only the gpt-4 models are considered.

The config list may look like the following:
```python
config_list = [
    {
        'model': 'gpt-4-1106-preview',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-02-15-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-02-15-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways if you prefer, e.g., loading from a YAML file.

In [4]:
import os
from typing import Dict, Optional, Union
import chromadb
from chromadb.config import Settings
import pickle
from autogen.agentchat.assistant_agent import ConversableAgent
from autogen.agentchat.contrib.capabilities.agent_capability import AgentCapability
from autogen.agentchat.contrib.text_analyzer_agent import TextAnalyzerAgent
from autogen.agentchat.conversable_agent import colored

class Teachability(AgentCapability):
    """
    Teachability uses a vector database to give an agent the ability to remember user teachings,
    where the user is any caller (human or not) sending messages to the teachable agent.
    Teachability is designed to be composable with other agent capabilities.
    To make any conversable agent teachable, instantiate both the agent and the Teachability class,
    then pass the agent to teachability.add_to_agent(agent).
    Note that teachable agents in a group chat must be given unique path_to_db_dir values.

    When adding Teachability to an agent, the following are modified:
    - The agent's system message is appended with a note about the agent's new ability.
    - A hook is added to the agent's `process_last_received_message` hookable method,
    and the hook potentially modifies the last of the received messages to include earlier teachings related to the message.
    Added teachings do not propagate into the stored message history.
    If new user teachings are detected, they are added to new memos in the vector database.
    """

    def __init__(
        self,
        verbosity: Optional[int] = 0,
        reset_db: Optional[bool] = False,
        path_to_db_dir: Optional[str] = "./tmp/teachable_agent_db",
        recall_threshold: Optional[float] = 1.5,
        max_num_retrievals: Optional[int] = 10,
        llm_config: Optional[Union[Dict, bool]] = None,
    ):
        """
        Args:
            verbosity (Optional, int): # 0 (default) for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
            reset_db (Optional, bool): True to clear the DB before starting. Default False.
            path_to_db_dir (Optional, str): path to the directory where this particular agent's DB is stored. Default "./tmp/teachable_agent_db"
            recall_threshold (Optional, float): The maximum distance for retrieved memos, where 0.0 is exact match. Default 1.5. Larger values allow more (but less relevant) memos to be recalled.
            max_num_retrievals (Optional, int): The maximum number of memos to retrieve from the DB. Default 10.
            llm_config (dict or False): llm inference configuration passed to TextAnalyzerAgent.
                If None, TextAnalyzerAgent uses llm_config from the teachable agent.
        """
        self.verbosity = verbosity
        self.path_to_db_dir = path_to_db_dir
        self.recall_threshold = recall_threshold
        self.max_num_retrievals = max_num_retrievals
        self.llm_config = llm_config

        self.analyzer = None
        self.teachable_agent = None

        # Create the memo store.
        self.memo_store = MemoStore(self.verbosity, reset_db, self.path_to_db_dir)

    def add_to_agent(self, agent: ConversableAgent):
        """Adds teachability to the given agent."""
        self.teachable_agent = agent

        # Register a hook for processing the last message.
        agent.register_hook(hookable_method="process_last_received_message", hook=self.process_last_received_message)

        # Was an llm_config passed to the constructor?
        if self.llm_config is None:
            # No. Use the agent's llm_config.
            self.llm_config = agent.llm_config
        assert self.llm_config, "Teachability requires a valid llm_config."

        # Create the analyzer agent.
        self.analyzer = TextAnalyzerAgent(llm_config=self.llm_config)

        # Append extra info to the system message.
        agent.update_system_message(
            agent.system_message
            + "\nYou've been given the special ability to remember user teachings from prior conversations."
        )

    def prepopulate_db(self):
        """Adds a few arbitrary memos to the DB."""
        self.memo_store.prepopulate()

    def process_last_received_message(self, text):
        """
        Appends any relevant memos to the message text, and stores any apparent teachings in new memos.
        Uses TextAnalyzerAgent to make decisions about memo storage and retrieval.
        """

        # Try to retrieve relevant memos from the DB.
        expanded_text = text
        if self.memo_store.last_memo_id > 0:
            expanded_text = self._consider_memo_retrieval(text)

        # Try to store any user teachings in new memos to be used in the future.
        self._consider_memo_storage(text)

        # Return the (possibly) expanded message text.
        return expanded_text

    # This is the role specific listening in the conversation that is required. In the case of the Wealth Advisor
    # 1. Listen for information about how to solve a problem - this is generic for all role
    # 2. Listen for information about the client. This is specific to a Wealth Advisor who serves clients
    # 3. Listen for cues about how to serve the client. In this case we want to understand if we have disappointed the client so as to never disappoint the client in the future

    def _consider_memo_storage(self, comment):
        # Decides whether to store something from one user comment in the DB.
        memo_added = False

        # Check for a problem-solution pair.
        response = self._analyze(
            comment,
            selfanalyze1,
        )
        if "yes" in response.lower():
            # Can we extract advice?
            advice = self._analyze(
                comment,
                selfanalyze1A,
            )
            if "none" not in advice.lower():
                # Yes. Extract the task.
                task = self._analyze(
                    comment,
                    selfanalyze1B,
                )
                # Generalize the task.
                general_task = self._analyze(
                    task,
                    selfanalyze1C,
                )
                # Add the task-advice (problem-solution) pair to the vector DB.
                if self.verbosity >= 1:
                    print(colored("\nREMEMBER THIS TASK-ADVICE PAIR", "light_yellow"))
                self.memo_store.add_input_output_pair(general_task, advice)
                memo_added = True

        # Check for information to be learned.
        response = self._analyze(
            comment,
            "Does the TEXT contain information that could be committed to memory? Answer with just one word, yes or no.",
        )
        if "yes" in response.lower():
            # Yes. What question would this information answer?
            question = self._analyze(
                comment,
                "Imagine that the user forgot this information in the TEXT. How would they ask you for this information? Include no other text in your response.",
            )
            # Extract the information.
            answer = self._analyze(
                comment, "Copy the information from the TEXT that should be committed to memory. Add no explanation."
            )
            # Add the question-answer pair to the vector DB.
            if self.verbosity >= 1:
                print(colored("\nREMEMBER THIS QUESTION-ANSWER PAIR", "light_yellow"))
            self.memo_store.add_input_output_pair(question, answer)
            memo_added = True

        # Were any memos added?
        if memo_added:
            # Yes. Save them to disk.
            self.memo_store._save_memos()
            
        # Decides whether to store something from one user comment in the DB.
        memo_added = False

        # Check for a problem-solution pair.
        response = self._analyze(
            comment,
            selfanalyze2,
        )
        if "yes" in response.lower():
            # Can we extract advice?
            advice = self._analyze(
                comment,
                selfanalyze2A,
            )
            if "none" not in advice.lower():
                # Yes. Extract the task.
                task = self._analyze(
                    comment,
                    selfanalyze2B,
                )
                # Generalize the task.
                general_task = self._analyze(
                    task,
                    selfanalyze2C,
                )
                # Add the task-advice (problem-solution) pair to the vector DB.
                if self.verbosity >= 1:
                    print(colored("\nREMEMBER THIS TASK-ADVICE PAIR", "light_yellow"))
                self.memo_store.add_input_output_pair(general_task, advice)
                memo_added = True

        # Check for information to be learned.
        response = self._analyze(
            comment,
            "Does the TEXT contain information that could be committed to memory? Answer with just one word, yes or no.",
        )
        if "yes" in response.lower():
            # Yes. What question would this information answer?
            question = self._analyze(
                comment,
                "Imagine that the user forgot this information in the TEXT. How would they ask you for this information? What would the steps be to avoid the disapproval in the future? Include no other text in your response.",
            )
            # Extract the information.
            answer = self._analyze(
                comment, "Copy the information from the TEXT that should be committed to memory. Add no explanation."
            )
            # Add the question-answer pair to the vector DB.
            if self.verbosity >= 1:
                print(colored("\nREMEMBER THIS QUESTION-ANSWER PAIR", "light_yellow"))
            self.memo_store.add_input_output_pair(question, answer)
            memo_added = True

        # Were any memos added?
        if memo_added:
            # Yes. Save them to disk.
            self.memo_store._save_memos()


    def _consider_memo_retrieval(self, comment):
        """Decides whether to retrieve memos from the DB, and add them to the chat context."""

        # First, use the comment directly as the lookup key.
        if self.verbosity >= 1:
            print(colored("\nLOOK FOR RELEVANT MEMOS, AS QUESTION-ANSWER PAIRS", "light_yellow"))
        memo_list = self._retrieve_relevant_memos(comment)

        # Next, if the comment involves a task, then extract and generalize the task before using it as the lookup key.
        response = self._analyze(
            comment,
            "Does any part of the TEXT ask the agent to perform a task or solve a problem? Answer with just one word, yes or no.",
        )
        if "yes" in response.lower():
            if self.verbosity >= 1:
                print(colored("\nLOOK FOR RELEVANT MEMOS, AS TASK-ADVICE PAIRS", "light_yellow"))
            # Extract the task.
            task = self._analyze(
                comment, "Copy just the task from the TEXT, then stop. Don't solve it, and don't include any advice."
            )
            # Generalize the task.
            general_task = self._analyze(
                task,
                "Summarize very briefly, in general terms, the type of task described in the TEXT. Leave out details that might not appear in a similar problem.",
            )
            # Append any relevant memos.
            memo_list.extend(self._retrieve_relevant_memos(general_task))

        # De-duplicate the memo list.
        memo_list = list(set(memo_list))

        # Append the memos to the text of the last message.
        return comment + self._concatenate_memo_texts(memo_list)

    def _retrieve_relevant_memos(self, input_text):
        """Returns semantically related memos from the DB."""
        memo_list = self.memo_store.get_related_memos(
            input_text, n_results=self.max_num_retrievals, threshold=self.recall_threshold
        )

        if self.verbosity >= 1:
            # Was anything retrieved?
            if len(memo_list) == 0:
                # No. Look at the closest memo.
                print(colored("\nTHE CLOSEST MEMO IS BEYOND THE THRESHOLD:", "light_yellow"))
                self.memo_store.get_nearest_memo(input_text)
                print()  # Print a blank line. The memo details were printed by get_nearest_memo().

        # Create a list of just the memo output_text strings.
        memo_list = [memo[1] for memo in memo_list]
        return memo_list

    def _concatenate_memo_texts(self, memo_list):
        """Concatenates the memo texts into a single string for inclusion in the chat context."""
        memo_texts = ""
        if len(memo_list) > 0:
            info = "\n# Memories that might help\n"
            for memo in memo_list:
                info = info + "- " + memo + "\n"
            if self.verbosity >= 1:
                print(colored("\nMEMOS APPENDED TO LAST MESSAGE...\n" + info + "\n", "light_yellow"))
            memo_texts = memo_texts + "\n" + info
        return memo_texts

    def _analyze(self, text_to_analyze, analysis_instructions):
        """Asks TextAnalyzerAgent to analyze the given text according to specific instructions."""
        self.analyzer.reset()  # Clear the analyzer's list of messages.
        self.teachable_agent.send(
            recipient=self.analyzer, message=text_to_analyze, request_reply=False, silent=(self.verbosity < 2)
        )  # Put the message in the analyzer's list.
        self.teachable_agent.send(
            recipient=self.analyzer, message=analysis_instructions, request_reply=True, silent=(self.verbosity < 2)
        )  # Request the reply.
        return self.teachable_agent.last_message(self.analyzer)["content"]


class MemoStore:
    """
    Provides memory storage and retrieval for a teachable agent, using a vector database.
    Each DB entry (called a memo) is a pair of strings: an input text and an output text.
    The input text might be a question, or a task to perform.
    The output text might be an answer to the question, or advice on how to perform the task.
    Vector embeddings are currently supplied by Chroma's default Sentence Transformers.
    """

    def __init__(self, verbosity, reset, path_to_db_dir):
        """
        Args:
            - verbosity (Optional, int): 1 to print memory operations, 0 to omit them. 3+ to print memo lists.
            - path_to_db_dir (Optional, str): path to the directory where the DB is stored.
        """
        self.verbosity = verbosity
        self.path_to_db_dir = path_to_db_dir

        # Load or create the vector DB on disk.
        settings = Settings(
            anonymized_telemetry=False, allow_reset=True, is_persistent=True, persist_directory=path_to_db_dir
        )
        self.db_client = chromadb.Client(settings)
        self.vec_db = self.db_client.create_collection("memos", get_or_create=True)  # The collection is the DB.

        # Load or create the associated memo dict on disk.
        self.path_to_dict = os.path.join(path_to_db_dir, "uid_text_dict.pkl")
        self.uid_text_dict = {}
        self.last_memo_id = 0
        if (not reset) and os.path.exists(self.path_to_dict):
            print(colored("\nLOADING MEMORY FROM DISK", "light_green"))
            print(colored("    Location = {}".format(self.path_to_dict), "light_green"))
            with open(self.path_to_dict, "rb") as f:
                self.uid_text_dict = pickle.load(f)
                self.last_memo_id = len(self.uid_text_dict)
                if self.verbosity >= 3:
                    self.list_memos()

        # Clear the DB if requested.
        if reset:
            self.reset_db()

    def list_memos(self):
        """Prints the contents of MemoStore."""
        print(colored("LIST OF MEMOS", "light_green"))
        for uid, text in self.uid_text_dict.items():
            input_text, output_text = text
            print(
                colored(
                    "  ID: {}\n    INPUT TEXT: {}\n    OUTPUT TEXT: {}".format(uid, input_text, output_text),
                    "light_green",
                )
            )

    def _save_memos(self):
        """Saves self.uid_text_dict to disk."""
        with open(self.path_to_dict, "wb") as file:
            pickle.dump(self.uid_text_dict, file)

    def reset_db(self):
        """Forces immediate deletion of the DB's contents, in memory and on disk."""
        print(colored("\nCLEARING MEMORY", "light_green"))
        self.db_client.delete_collection("memos")
        self.vec_db = self.db_client.create_collection("memos")
        self.uid_text_dict = {}
        self._save_memos()

    def add_input_output_pair(self, input_text, output_text):
        """Adds an input-output pair to the vector DB."""
        self.last_memo_id += 1
        self.vec_db.add(documents=[input_text], ids=[str(self.last_memo_id)])
        self.uid_text_dict[str(self.last_memo_id)] = input_text, output_text
        if self.verbosity >= 1:
            print(
                colored(
                    "\nINPUT-OUTPUT PAIR ADDED TO VECTOR DATABASE:\n  ID\n    {}\n  INPUT\n    {}\n  OUTPUT\n    {}\n".format(
                        self.last_memo_id, input_text, output_text
                    ),
                    "light_yellow",
                )
            )
        if self.verbosity >= 3:
            self.list_memos()

    def get_nearest_memo(self, query_text):
        """Retrieves the nearest memo to the given query text."""
        results = self.vec_db.query(query_texts=[query_text], n_results=1)
        uid, input_text, distance = results["ids"][0][0], results["documents"][0][0], results["distances"][0][0]
        input_text_2, output_text = self.uid_text_dict[uid]
        assert input_text == input_text_2
        if self.verbosity >= 1:
            print(
                colored(
                    "\nINPUT-OUTPUT PAIR RETRIEVED FROM VECTOR DATABASE:\n  INPUT1\n    {}\n  OUTPUT\n    {}\n  DISTANCE\n    {}".format(
                        input_text, output_text, distance
                    ),
                    "light_yellow",
                )
            )
        return input_text, output_text, distance

    def get_related_memos(self, query_text, n_results, threshold):
        """Retrieves memos that are related to the given query text within the specified distance threshold."""
        if n_results > len(self.uid_text_dict):
            n_results = len(self.uid_text_dict)
        results = self.vec_db.query(query_texts=[query_text], n_results=n_results)
        memos = []
        num_results = len(results["ids"][0])
        for i in range(num_results):
            uid, input_text, distance = results["ids"][0][i], results["documents"][0][i], results["distances"][0][i]
            if distance < threshold:
                input_text_2, output_text = self.uid_text_dict[uid]
                assert input_text == input_text_2
                if self.verbosity >= 1:
                    print(
                        colored(
                            "\nINPUT-OUTPUT PAIR RETRIEVED FROM VECTOR DATABASE:\n  INPUT1\n    {}\n  OUTPUT\n    {}\n  DISTANCE\n    {}".format(
                                input_text, output_text, distance
                            ),
                            "light_yellow",
                        )
                    )
                memos.append((input_text, output_text, distance))
        return memos

    def prepopulate(self):
        """Adds a few arbitrary examples to the vector DB, just to make retrieval less trivial."""
        if self.verbosity >= 1:
            print(colored("\nPREPOPULATING MEMORY", "light_green"))
        examples = []
        examples.append({"text": "When I say papers I mean research papers, which are typically pdfs.", "label": "yes"})
        examples.append({"text": "Please verify that each paper you listed actually uses langchain.", "label": "no"})
        examples.append({"text": "Tell gpt the output should still be latex code.", "label": "no"})
        examples.append({"text": "Hint: convert pdfs to text and then answer questions based on them.", "label": "yes"})
        examples.append(
            {"text": "To create a good PPT, include enough content to make it interesting.", "label": "yes"}
        )
        examples.append(
            {
                "text": "No, for this case the columns should be aspects and the rows should be frameworks.",
                "label": "no",
            }
        )
        examples.append({"text": "When writing code, remember to include any libraries that are used.", "label": "yes"})
        examples.append({"text": "Please summarize the papers by Eric Horvitz on bounded rationality.", "label": "no"})
        examples.append({"text": "Compare the h-index of Daniel Weld and Oren Etzioni.", "label": "no"})
        examples.append(
            {
                "text": "Double check to be sure that the columns in a table correspond to what was asked for.",
                "label": "yes",
            }
        )
        for example in examples:
            self.add_input_output_pair(example["text"], example["label"])
        self._save_memos()


## Construct Agents
For this walkthrough, we start by creating a teachable agent and resetting its memory store. This deletes any memories from prior conversations that may be stored on disk.

In [5]:
# Start by instantiating any agent that inherits from ConversableAgent.

# This is the definition of the Wealth Advisor Agent
# We define the Knowledge, Experience, Skills and Tools available to the role


role_teachable_agent = autogen.AssistantAgent(
    name=def_role_name,  # The name is flexible, but should not contain spaces to work in group chat.
    llm_config={"config_list": config_list, "timeout": 240, "cache_seed": None},  # Disable caching.
    system_message=teachable_agent_system_message
)

# Instantiate the Teachability capability. Its parameters are all optional.
teachability = Teachability(
    verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    reset_db=True,
    path_to_db_dir="./teachability_db",
    recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
)

# Now add the Teachability capability to the agent.
teachability.add_to_agent(role_teachable_agent)

# Instantiate a UserProxyAgent to represent the user. But in this notebook, all user input will be simulated.
user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    code_execution_config={
        "use_docker": False,
        "last_n_messages": 1,
        "work_dir": "tasks",
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    system_message=user_agent_system_message
)

# Create the mentor agent. In this case we'll create a virtual Warren Buffet who can analyze the conversation and provide feedback to the Wealth Advisor that they can use in the future.

mentor = autogen.AssistantAgent(
    name="Mentor",
    llm_config={"config_list": config_list},
    system_message=mentor_agent_system_message,
)

# Create the personality agent. We use the Myers-Briggs definition to modulate the advisor.

personality = autogen.AssistantAgent(
    name="Personality",
    llm_config={"config_list": config_list},
    system_message=personality_agent_system_message,
)

# This reflection concept is overkill for what is needed. I can just filter each answer and provide advice.

# Within the framework we have the opportunity for reflection before giving an answer back to the client. We will use this opportunity to do 2 things:
# 1. Ensure that the text is in the second person and is clear and in simple language.
# 2. Make sure that it is presented in the personality that is best for the role. 

# def reflection_message(recipient, messages, sender, config):
#    print("Reflecting...", "yellow")
#    return f"{reflection1}"

# def personality_message(recipient, messages, sender, config):
#    print("Reflecting...", "yellow")
#    return f"{reflection2}"

# user.register_nested_chats(
#    [{"recipient": personality, "message": personality_message, "summary_method": "last_msg", "max_turns": 1},
#     {"recipient": mentor, "message": reflection_message, "summary_method": "last_msg", "max_turns": 1}],
#    trigger=role_teachable_agent,  # condition=my_condition,
#)

# Execute a test task

# res = user.initiate_chat(recipient=role_teachable_agent, message=task, max_turns=2, summary_method="last_msg")




CLEARING MEMORY


In [6]:
# Step 1: We create a basic profile for John Smith and his family

text = """
Your client is John Smith
  Sex: Male, 40 years old
  Home in Unionville, Ontario, Canada
  Family: Married to Jane Smith, 39 years old. Two daughters, Meaghan, 12 and Rachael, 10 years old
  Income: John Smith, $100,000 per year, Jane Smith, $80,000 per year
  Occupation: Teacher
  Risk Profile: Medium
  Car loan: $20,000
  No Life Insurance
  Mortgage: $800,000 on a $1,000,000 home
  Exercise once a week
  Plan for 2 family vacations per year
  Too much sedentary work

Review the data provided for John Smith and use the inputs. Create the set of recommendations you would provide for John Smith given the data provided. Think about how you would create the plan step by step and then create the recommendations.
"""

# We will ask the Augmentation Agent for feedback. We will then ask the Personality and Mentor Agents to weigh in and provide their feedback as well to improve our performance in the future

res = user.initiate_chat(role_teachable_agent, message=text, max_turns=1, clear_history=True)

user.initiate_chat(personality, message = "Rewrite this text in the specified personality " + res.summary, max_turns=1, clear_history=True)

user.initiate_chat(mentor, message = "Find general mentoring advice in this " + res.summary, max_turns=1, clear_history=True)

# Step 2: Retrospection - This could be done on every turn as well with a specialized agent. 

text = "The information provided about John Smith is incomplete. What additional insights could you provide for John Smith and what data would you need for those insights?"

user.initiate_chat(role_teachable_agent, message=text, max_turns=1, clear_history=True)

# Step 3: Provide additional facts about John Smith and his family

text = """
Here are some additional facts about John Smith's financial profile:
1. The children are in private school and it costs $40,000 per student per year in the school
2. Utilities and home taxes are $24,000 per year
3. John has $100,000 in his bank savings account
4. John has not invested in RRSP or RESP in the past
5. John has a work pension as a teacher
6. John has health coverage through his employer and has the provincial OHIP plan
7. John likes expensive cars and a luxury lifestyle that he may not be able to afford

Add this information to the customer profile and first step print out the new customer profile.

Create the set of recommendations you would provide for John Smith given the data provided. Think about how you would create the plan step by step and then create the recommendations
"""

user.initiate_chat(role_teachable_agent, message=text, max_turns=1, clear_history=True)

# Step 4: Add one additional fact about John Smith and his family

text = """
You have new facts about John Smith's financial profile. He has inherited $2,000,000 from his father and has put this in a GIC in the bank. Include this information in John Smith's profile. John needs advice on what to do with this money. Include this information in his financial profile and for his recommendations goimg forward.

In the form of attribute:value print out the profile for John Smith and his family 
"""

user.initiate_chat(role_teachable_agent, message=text, max_turns=1, clear_history=True)

# Step 5: Print the current state of the client profile learned so far 

user.initiate_chat(role_teachable_agent, message="In the form of attribute:value print out the profile for John Smith and his family", max_turns=1, clear_history=True)

# Step 6: An example where the client was disappointed. 

text = "The analysis is lacking in terms of the needs of the family cat, Patches, in addition to the children and family. Please recreate your recommendations. Patches is a 10 year old black and white cat and is considered a member of the family. Also John prefers information about the housing market in Markham going forward as he is considering purchasing a house there."

user.initiate_chat(role_teachable_agent, message=text, max_turns=1, clear_history=True)

# Step 7: Final

text = "Create the set of recommendations you would provide for John Smith and his family. Ensure you do not disappoint him. Think about how you would create the plan step by step and then create the recommendations"

user.initiate_chat(role_teachable_agent, message=text, max_turns=1, clear_history=True)

user.initiate_chat(personality, message = "Rewrite this text in the specified personality " + res.summary, max_turns=1, clear_history=True)

user.initiate_chat(mentor, message = "Find general mentoring advice in this " + res.summary, max_turns=1, clear_history=True)

# Step 8: Print out the current profile for the client

res = user.initiate_chat(role_teachable_agent, message="In the form of attribute:value print out the profile for John Smith and his family", max_turns=1, clear_history=True)

User (to WealthAdvisor):


Your client is John Smith
  Sex: Male, 40 years old
  Home in Unionville, Ontario, Canada
  Family: Married to Jane Smith, 39 years old. Two daughters, Meaghan, 12 and Rachael, 10 years old
  Income: John Smith, $100,000 per year, Jane Smith, $80,000 per year
  Occupation: Teacher
  Risk Profile: Medium
  Car loan: $20,000
  No Life Insurance
  Mortgage: $800,000 on a $1,000,000 home
  Exercise once a week
  Plan for 2 family vacations per year
  Too much sedentary work

Review the data provided for John Smith and use the inputs. Create the set of recommendations you would provide for John Smith given the data provided. Think about how you would create the plan step by step and then create the recommendations.


--------------------------------------------------------------------------------
WealthAdvisor (to User):

To provide John Smith with a comprehensive financial plan, we'll consider various aspects including budgeting, debt management, investment strat

In [7]:
# Final Questions for Bloom's Taxonomy Evaluation

text = "What do you remember about John Smith?"
user.initiate_chat(role_teachable_agent, message=text, max_turns=1, clear_history=True)

text = "Can you describe his financial profile?"
user.initiate_chat(role_teachable_agent, message=text, max_turns=1, clear_history=True)

text = "How would uou describe his financial situation?"
user.initiate_chat(role_teachable_agent, message=text, max_turns=1, clear_history=True)

text = "What recommendations would you make about his financial situation"
user.initiate_chat(role_teachable_agent, message=text, max_turns=1, clear_history=True)

text = "What additional information would you like to have about John Smith and what additional insights could you provide?"
user.initiate_chat(role_teachable_agent, message=text, max_turns=1, clear_history=True)

text = "Create a year by year plan for the next 3 years for John Smith and his family to improve their financial health and meet their financial goals"
user.initiate_chat(role_teachable_agent, message=text, max_turns=1, clear_history=True)


User (to WealthAdvisor):

What do you remember about John Smith?

--------------------------------------------------------------------------------
WealthAdvisor (to User):

Great, let’s consolidate the financial and personal profile of John Smith further for a more comprehensive understanding. Here’s a combined overview based on your inputs:

### Financial Profile
1. **Income:**
   - John Smith: $100,000 per year
   - Jane Smith: $80,000 per year 

2. **Expenses:**
   - Private School: $40,000 per student per year ($80,000 annually for two children)
   - Utilities and Home Taxes: $24,000 per year
   - Car Loan: $20,000 remaining
   - Lifestyle: Preferences for expensive cars and luxury items (assume high discretionary spending)
   - Family Vacations: Planned for 2 vacations per year (est. $10,000-$15,000 annually)

3. **Savings and Investments:**
   - Bank Savings Account: $100,000
   - GIC from Inheritance: $2,000,000
   - No current investments in RRSP or RESP

4. **Liabilities:**
  

ChatResult(chat_id=None, chat_history=[{'content': 'Create a year by year plan for the next 3 years for John Smith and his family to improve their financial health and meet their financial goals', 'role': 'assistant'}, {'content': "Here's a detailed year-by-year financial plan for John Smith and his family for the next three years, considering their financial profile, goals, and needs.\n\n### Year 1\n\n#### Income and Investments\n1. **Income Allocation**\n   - John's salary: $100,000\n   - Jane's salary: $80,000\n   - Total Income: $180,000\n\n2. **Investment in RRSP and RESP**\n   - RRSP Contributions: Aim to contribute $18,000 (10% of gross income)\n   - RESP Contributions for both daughters: $5,000 per child = $10,000\n\n3. **Review GICs**\n   - Analyze the $2,000,000 in GICs and evaluate alternatives like diversified investment portfolios to achieve higher returns while managing risk.\n\n#### Expenses\n1. **Private School Costs**\n   - $80,000 for both daughters\n\n2. **Utilities 